# Pair A/B Testing
## Ejercicio 1

Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

**Objetivo del ejercicio:** Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

## Pasos que debemos seguir:

1. **Preparación de los datos:**
   Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import chi2_contingency

In [18]:
# Ponemos este código para que se vean el máximo de las filas 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('marketing_AB.csv')
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [5]:
df.columns

Index(['Unnamed: 0', 'user id', 'test group', 'converted', 'total ads',
       'most ads day', 'most ads hour'],
      dtype='object')

In [6]:
df.describe()

,Unnamed: 0,user id,total ads,most ads hour
count,588101.000000,5.881010e+05,588101.000000,588101.000000
mean,294050.000000,1.310692e+06,24.820876,14.469061
std,169770.279667,2.022260e+05,43.715181,4.834634
min,0.000000,9.000000e+05,1.000000,0.000000
25%,147025.000000,1.143190e+06,4.000000,11.000000
50%,294050.000000,1.313725e+06,13.000000,14.000000
75%,441075.000000,1.484088e+06,27.000000,18.000000
max,588100.000000,1.654483e+06,2065.000000,23.000000


In [7]:
# Creamos un bucle for para sacar los valores únicos de cada columna
for column in df.columns:
        print(f"Columna: {column}")
        print(df[column].unique())

        print("\n--------------------\n")

Columna: Unnamed: 0
[     0      1      2 ... 588098 588099 588100]

--------------------

Columna: user id
[1069124 1119715 1144181 ... 1038442 1496395 1237779]

--------------------

Columna: test group
['ad' 'psa']

--------------------

Columna: converted
[False  True]

--------------------

Columna: total ads
[ 130   93   21  355  276  734  264   17  142  209   47   61   40   20
    9   64   26  248   73  281  389  136   87   46    3  114  175  119
  313   89  364   24  112   49   27  274   45  177   90  265   71   31
  588   56  108  103  239   63   66  111   74   14  258 1057  137  131
  124   13   37  738  199  211   98   19  127   18  126  197   77   60
   95   97   22  117   41   57  593   28   85   53   43  133   33 1328
    7    6   62  247  464   99  233  139  407  323   76  395  166   34
  246  653  122   32  386   44  138  109   39  159   30   59  421  446
  555  105  128  310  163   10  290  164   79  312   16  161  505  640
  118  266  456   80  100  983   36    1    5

In [8]:
# Buscamos si hay algún nulo en el DataFrame 
nulos = df.isnull().sum() 
print("\nConteo de valores nulos por columna:") 
print(nulos) 


Conteo de valores nulos por columna:
Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64


In [9]:
# Identificar y contar filas duplicadas 
duplicados = df.duplicated().sum() 
print("\nNúmero de filas duplicadas:") 
print(duplicados)


Número de filas duplicadas:
0


In [11]:
# Quitamos 'Unnamed: 0'
df = df.drop(columns = ['Unnamed: 0'])

In [12]:
# Imprimimos sólo 2 filas para ver si ha resultado el cambio 
df.head(2)

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22


# 2. **Definición de las hipótesis:**
   Plantea una `hipótesis nula (H0)` y una `hipótesis alternativa (H1)` para el experimento (en este caso, ya están definidas):

   - **Hipótesis nula (H0):** No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".
   - **Hipótesis alternativa (H1):** Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

_ _ _

1. **Hipótesis nula (H0):** Esta hipótesis establece que no hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa". 

2. **Hipótesis alternativa (H1):** La hipótesis alternativa sugiere que existe una diferencia significativa en la tasa de conversión entre los grupos "ad" y "psa".

En resumen, estamos evaluando si el nuevo diseño de anuncio ("ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño anterior ("psa"). 

# 3. **Cálculo de la tasa de conversión:**
   - Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.
   - Realiza el mismo cálculo para el grupo "psa".

NOTA: La `tasa de conversión` (Conversion Rate en inglés, CR) es un indicador clave en marketing y ventas que mide el porcentaje de visitantes de un sitio web u otros receptores de un mensaje de marketing que realizan la acción deseada, como realizar una compra, completar un formulario o suscribirse a un boletín informativo. Se calcula dividiendo el número de conversiones por el número total de visitantes o receptores.

In [17]:
# Creamos una función
def calcular_tasa_conversion(grupo):
  """
  Calcula la tasa de conversión para un grupo específico.

  Args:
    grupo: El nombre del grupo ("ad" o "psa").

  Returns:
    La tasa de conversión como un porcentaje.
  """
  # Filtra el DataFrame para obtener solo el grupo especificado
  df_grupo = df[df["test group"] == grupo]

  # Cuenta el número de conversiones en el grupo filtrado
  conversiones = df_grupo["converted"].sum()

  # Suma el número total de usuarios en el grupo filtrado
  total_usuarios = df_grupo["total ads"].sum()

  # Calcula la tasa de conversión como porcentaje
  tasa_conversion = (conversiones / total_usuarios) * 100

  # Devuelve la tasa de conversión calculada
  return tasa_conversion

# Tasa de conversión para el grupo "ad"
tasa_conversion_ad = calcular_tasa_conversion("ad")
print(f"Tasa de conversión para el grupo 'ad': {tasa_conversion_ad:.2f}%")

# Tasa de conversión para el grupo "psa"
tasa_conversion_psa = calcular_tasa_conversion("psa")
print(f"Tasa de conversión para el grupo 'psa': {tasa_conversion_psa:.2f}%")

Tasa de conversión para el grupo 'ad': 0.10%
Tasa de conversión para el grupo 'psa': 0.07%


# 4. **Prueba de hipótesis:**
   Utiliza una prueba estadística, como la prueba t de Student o la prueba chi-cuadrado, para comparar las tasas de conversión entre los grupos "ad" y "psa".

In [16]:
# Suponiendo que el DataFrame se llama "df"
contingency_table = pd.crosstab(df["test group"], df["converted"])
# Crea una tabla de contingencia para mostrar la distribución de conversiones por grupo
print(contingency_table)

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table.values)
# Calcula el chi-cuadrado, p-valor, grados de libertad y valores esperados
print(f"Chi-squared statistic: {chi2_stat:.2f}")  # Imprime el estadístico chi-cuadrado
print(f"p-value: {p_value:.4f}")  # Imprime el p-valor


converted    False   True
test group               
ad          550154  14423
psa          23104    420
Chi-squared statistic: 54.01
p-value: 0.0000


# 5. **Análisis de resultados:**
   - Si el valor p es menor que un umbral predefinido (por ejemplo, 0.05), rechazamos la hipótesis nula y concluimos que hay una diferencia significativa en las tasas de conversión entre los grupos.
   - Si el valor p es mayor que el umbral, no podemos rechazar la hipótesis nula y concluimos que no hay evidencia suficiente para afirmar que hay una diferencia significativa.

`Con lo que rechazamos la hipótesis nula y concluimos que existe una diferecia significativa`.